<a href="https://www.kaggle.com/code/mcsadri/linear-regresssion?scriptVersionId=127749060" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Linear Regression with sklearn

**Author**: [Manuch S.](https://github.com/mcsadri)  
**Input**: [2,2k+ Scotch Whisky Reviews](https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews): a dataset including 2,2k+ Scotch Whisky reviews (originally collected from the Whisky Advocate database.

#### Overview

Performing exploratory analysis by using Linear Regressions on a Kaggle data set.

#### Feature Tasks and Requirements

- [ ] Select a Kaggle data set that is suitable for Linear Regression.
    - Note make sure the data set is stored as csv file/s.
    - Data set must be continous values appropriate for a Linear Regression. If you’re not sure then ask Instructor or TA.
- [ ] Load the data you receive into a Pandas DataFrame.
- [ ] Show the first five rows of the data set.
- [ ] Show the description and the info of the data set.
- [ ] Ensure that any date columns have been cast into a datetime object in your DataFrame.
- [ ] Using a regression model, split your data into train and test portions.
- [ ] Fit your training split to the regression model.
- [ ] Show your regression model’s score.
- [ ] Draw at least three conclusions from your regression model.
- [ ] Your notebook should be clutter free and polished.

#### UAT

No acceptance tests today, but Notebook should clearly display its conclusions.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/22000-scotch-whisky-reviews/scotch_review.csv


### Read data file and get info() and shape() for the raw DataFrame

In [2]:
raw_df = pd.read_csv("/kaggle/input/22000-scotch-whisky-reviews/scotch_review.csv")
print("The raw DataFrame's information is:")
print("")
raw_df.info()
print("")
print("")
print(f"The raw DataFrame's shape is:, {np.shape(raw_df)}")
print()

The raw DataFrame's information is:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2247 non-null   int64 
 1   name          2247 non-null   object
 2   category      2247 non-null   object
 3   review.point  2247 non-null   int64 
 4   price         2247 non-null   object
 5   currency      2247 non-null   object
 6   description   2247 non-null   object
dtypes: int64(2), object(5)
memory usage: 123.0+ KB


The raw DataFrame's shape is:, (2247, 7)



### Validate which currencies are present in the DataFrame (so we can scrub any non-$ later, if needed)

In [3]:
# publisher_counts = df['Publisher'].value_counts().sort_values(ascending=False)
currency_types = raw_df['currency'].value_counts().sort_values(ascending=False)
currency_types

$    2247
Name: currency, dtype: int64

### Collect only the DataFrame series needed for linear regression modeling and prediction (i,e., review.point, price, currency)

In [4]:
working_df = raw_df[['review.point', 'price']]
working_df
# reviews_prices['price'].astype(str).astype(int)
# reviews_prices_filtered = reviews_prices['price'].select_dtypes(include = "number")

df_p = raw_df.price.str.replace("[({',$qwertyuioplkjhgfdsazxcvbnm%:]", " ")
# df_p.head(10)

# df_price=df_p.astype(float)
df_p.head(50)


# df_price=df_p.convert_objects(convert_numeric=True)
# df_price.sample()


# df_p=raw.price.str.replace("[({',$qwertyuioplkjhgfdsazxcvbnm%:]", " ")
# df_price=df_p.convert_objects(convert_numeric=True)
# df_price.sample()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


0                        225
1                    4500.00
2                   13500.00
3                        325
4                        160
5                      85.00
6                    6250.00
7                   11000.00
8                    1500.00
9                       3360
10                    750.00
11                   1500.00
12                      3108
13                    105.00
14                       120
15                   3500.00
16                     70.00
17                  20000.00
18                     70.00
19     15 000     60 000/   
20                     26650
21                    400.00
22                    200.00
23                    400.00
24                    455.00
25                       750
26                    200.00
27                    460.00
28                   2525.00
29                   1250.00
30                    280.00
31                    500.00
32                    215.00
33                    300.00
34            